<h2 align=center> Facial Expression Recognition</h2>

## Importing Libraries

In [ ]:
!pip install utils
!pip install livelossplot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

## Class for loading model and weights

In [ ]:
from tensorflow.keras.models import model_from_json
import numpy as np

import tensorflow as tf


class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]

    def __init__(self, model_file):
      
        self.loaded_model = load_model(model_file)

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]


## Getting frames and doing prediction

In [ ]:
  
import cv2
import numpy as np

facec = cv2.CascadeClassifier('Model weights/haarcascade_frontalface_default.xml')

model_basic = FacialExpressionModel("Model weights/basic_model.h5")
model_resnet = FacialExpressionModel("Model weights/RESNET_model.h5")

font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self, model_type):
        self.video = cv2.VideoCapture(0)
        self.model_type = model_type

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            if self.model_type == "RESNET":
                rgb_img = np.zeros_like(fr)
                rgb_img[:,:,0] = gray_fr
                rgb_img[:,:,1] = gray_fr
                rgb_img[:,:,2] = gray_fr   
                
                fc = rgb_img[y:y+h, x:x+w,:]
                roi = cv2.resize(fc, (224, 224))
                
                pred = model_resnet.predict_emotion(roi[np.newaxis, :, :])
            else:
                fc = gray_fr[y:y+h, x:x+w]
                
                roi = cv2.resize(fc, (48, 48))
                pred = model_basic.predict_emotion(roi[np.newaxis, :, :, np.newaxis])
           
            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr

## Function for showing output video

In [ ]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

## Running the code

In [ ]:
## Select model to use
gen(VideoCamera(model_type= ''))